# GStar and interference contributions

## Generate histograms

In [17]:
import zdb
import glob
import os
import oyaml as yaml
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dftools
import scipy
import pysge
from tqdm.auto import tqdm

plt.style.use('cms')
plt.rcParams.update({
    "figure.dpi": 150,
    "figure.figsize": (4, 3),
    "lines.markersize": 3,
    "errorbar.capsize": 2,
    "font.size": 10.,
})

In [18]:
def generate_yaml(hists, filepath, tables={"central": "Events"}, systs=[]):
    hists_def = {
        "_".join(k): [
            {"table": "df: '{table_name}'"},
            {"varname0": "df: '{}'".format(k[0])},
            {"varname1": "df: '{}'".format(k[1])},
            {"varname2": "df: '{}'".format(k[2])},
            {"selection": "df: '{selection_name}'"},
            {"parent": "df: df.parent"},
            {"mass_window": "df: (df.GenPartBoson_mass>=71.) & (df.GenPartBoson_mass<111.)"},
            {"large_pt": "df: df.GenPartBoson_pt>=200."},
            {"eta_restricted": "df: (np.abs(df.LeadGenLepCandidate_pt)<2.4) & (np.abs(df.SecondGenLepCandidate_pt)<2.4)"},
            {"binvar0": "df: df.{}_bin".format(k[0])},
            {"binvar1": "df: df.{}_bin".format(k[1])},
            {"binvar2": "df: df.{}_bin".format(k[2])},
            {"count": "df: 1."},
            {"sum_w": "df: {weight}"},
            {"sum_ww": "df: df.sum_w**2"},
        ] for k in hists
    }
    
    for key, hdef in hists_def.items():
        for label, vari in systs:
            hdef.extend([
                {"sum_w_{}".format(label): "df: df.sum_w*({})".format(vari)},
                {"sum_ww_{}".format(label): "df: (df.sum_w*({}))**2".format(vari)},
            ])
    
    cutflows = {
        "DoubleLepton": {
            "selection_name": "DoubleLepton",
            "selection": "GenPartBoson_mass>=0",
            "weight": "df.WeightXsLumi",
            "hists": list(hists_def.keys()),
        },
    }

    cfg_eval_bins = [
        {"GenPartBoson_mass_bin": "df: np.floor(df.GenPartBoson_mass/1.)*1."},
        {"GenPartBoson_pt_bin": "df: np.floor(df.GenPartBoson_pt/5.)*5."},
        {"LeadGenLepCandidate_pt_bin": "df: np.floor(df.LeadGenLepCandidate_pt/5.)*5."},
        {"LeadGenLepCandidate_eta_bin": "df: np.floor(df.LeadGenLepCandidate_eta/0.1)*0.1"},
        {"SecondGenLepCandidate_pt_bin": "df: np.floor(df.SecondGenLepCandidate_pt/5.)*5."},
        {"SecondGenLepCandidate_eta_bin": "df: np.floor(df.SecondGenLepCandidate_eta/0.1)*0.1"},
        {"Generator_id1_bin": "df: np.floor(np.minimum(df.Generator_id1, df.Generator_id2)/1)*1"},
        {"Generator_id2_bin": "df: np.floor(np.maximum(df.Generator_id1, df.Generator_id2)/1)*1"},
    ]
    
    cfg = {
        "query": {
            "groupby": [
                "table", "varname0", "varname1", "varname2",
                "selection", "parent", "mass_window", "large_pt", "eta_restricted",
                "binvar0", "binvar1", "binvar2",
            ],
            "tables": tables,
            "aliases": {},
            "eval": cfg_eval_bins,
            "cutflows": cutflows,
            "hists": hists_def,
        },
        "files": sorted(p for p in glob.glob(filepath)),
    }
    
    return cfg

In [19]:
hists = [
    ("GenPartBoson_mass", "Generator_id1", "Generator_id2"),
    ("GenPartBoson_pt", "Generator_id1", "Generator_id2"),
    ("LeadGenLepCandidate_pt", "Generator_id1", "Generator_id2"),
    ("LeadGenLepCandidate_eta", "Generator_id1", "Generator_id2"),
    ("SecondGenLepCandidate_pt", "Generator_id1", "Generator_id2"),
    ("SecondGenLepCandidate_eta", "Generator_id1", "Generator_id2"),
]
systs = [
    ("alphasUp", "df.LHEPdfWeight101"),
    ("alphasDown", "df.LHEPdfWeight102"),
] + [
    ("lheScaleWeight{}".format(idx), "np.where(~(df.parent.str.contains('ZJetsTo') | df.parent.str.contains('WJetsTo') | df.parent.str.contains('DYJetsTo') | df.parent.str.contains('GStarJetsTo')), df.LHEScaleWeight{}, np.ones_like(df.LHEScaleWeight0))".format(idx))
    for idx in (0, 1, 3, 5, 7, 8)
] + [
    ("lhePdfWeight{}".format(idx), "df.LHEPdfWeight{}".format(idx))
    for idx in range(99)
]

cfg = generate_yaml(
    hists, "/vols/cms/sdb15/Analysis/ZinvWidth/databases/2019/06_Jun/14_gstar/MC/*.h5",
    systs=systs,
)
with open("configs/mc.yaml", "w") as f:
    yaml.dump(cfg, f, indent=4)

In [20]:
!~/Scripts/batch/QSTAT.py


 queue   |   free |   used |   total
---------+--------+--------+---------
 hep.q   |    373 |     67 |     440
 gpu.q   |      7 |      0 |       7
 fw.q    |      4 |      0 |       4

 user   |     hep.q |
        |   running | duration
--------+-----------+-----------------
 Brais  |        67 | 0 days 06:10:30



In [12]:
#zdb.modules.analyse(
#    "configs/mc.yaml",
#    output="hists_1d_gstar.h5:MCAggEvents",
#    mode='sge',
#    ncores=-1,
#    batch_opts="-q hep.q -l h_rt=3:0:0 -l h_vmem=12G",
#    chunksize=500_000,
#    merge_opts={"mode": "sge", "ncores": 10, "batch_opts": "-q hep.q"},
#)